In [1]:
import pandas as pd
import os
import re

In [2]:
d = pd.read_pickle('all_distilroberta_flickr8k_embeddings.pkl')
d.shape

(8091, 5)

In [3]:
directory = '.'
def rank_metrics(directory):
    best_median_rank_score = 1000000000
    best_hit_10_score = 0 
    best_hit_5_score = 0
    best_hit_1_score = 0
    best_mrr = 0
    median_rank_models = []
    median_rank = {}
    hit_at_1 = {}
    hit_at_5 = {}
    hit_at_10 = {}
    mrr_5 = {}
    mrr_10 = {}
    mrr_all = {}

    for filename in os.listdir(directory):
        if filename.endswith("ranking_results_8k.pkl"): 
            df = pd.read_pickle(os.path.join(directory, filename))
            for idx, data in df.iterrows():
                median_rank[idx] = data['median hit pos all']
                hit_at_1[idx] = data['rank 1']
                hit_at_5[idx] = data['rank 5']
                hit_at_10[idx] = data['rank 10']
                mrr_5[idx] = data['mrr 5']
                mrr_10[idx] = data['mrr 10']
                mrr_all[idx] = data['mrr all']
                print('\n Model: ', idx)
                print('median position:', data['median hit pos all'])
                print('hit@1:', data['rank 1'], ' - hit@5:', data['rank 5'], ' - hit@10:', data['rank 10'])
                print('MRR in all data: ', data['mrr all'])
                if data['median hit pos all']<best_median_rank_score:    # find best median rank
                    best_median_rank_score = data['median hit pos all']
                    best_median_rank_model = idx
                if data['rank 10']>best_hit_10_score:                    # find best hit@10
                    best_hit_10_score = data['rank 10']
                    best_hit_10_model = idx
                if data['rank 5']>best_hit_5_score:                      # find best hit@5
                    best_hit_5_score = data['rank 5']
                    best_hit_5_model = idx
                if data['rank 1']>best_hit_1_score:                      # find best hit@1
                    best_hit_1_score = data['rank 1']
                    best_hit_1_model = idx   
                # MRR values from 0 (worst) to 1 (best), higher => better
                if data['mrr all']>best_mrr:
                    best_mrr = data['mrr all']
                    best_mrr_model = idx
                
    for key in median_rank:
        val = median_rank[key]
        if val == best_median_rank_score:
            median_rank_models.append(key)   # keep only best-ranking models

    print('\n', 30*'*', 'Final Results', 30*'*')
    print('Best model(s) for median position:', [i for i in median_rank_models])
    print('Best model for Hit@10:', best_hit_10_model, '-', best_hit_10_score)
    print('Best model for Hit@5:', best_hit_5_model, '-', best_hit_5_score)
    print('Best model for Hit@1:', best_hit_1_model, '-', best_hit_1_score)
    print('Best model for MRR (global rank):', best_mrr_model, '-', best_mrr)
    
    return median_rank, hit_at_1, hit_at_5, hit_at_10, mrr_all
median_rank, hit_at_1, hit_at_5, hit_at_10, mrr_all = rank_metrics(directory)


 Model:  all_distilroberta
median position: 2.0
hit@1: 0.46990483252997156  - hit@5: 0.6971944135459152  - hit@10: 0.7754294895562971
MRR in all data:  0.5743539984492478

 Model:  paraphrase_albert_base
median position: 2.0
hit@1: 0.4775676677790137  - hit@5: 0.7167222840192807  - hit@10: 0.7956989247311828
MRR in all data:  0.586448437441523

 Model:  paraphrase_mpnet
median position: 2.0
hit@1: 0.49363490297861823  - hit@5: 0.7318007662835249  - hit@10: 0.8117661599307873
MRR in all data:  0.602188949781251

 Model:  stsb_roberta_large
median position: 3.0
hit@1: 0.367816091954023  - hit@5: 0.5693980966505995  - hit@10: 0.6575207020145841
MRR in all data:  0.4642690105047136

 Model:  multi_qa_mpnet_base_cos
median position: 3.0
hit@1: 0.3989618094178717  - hit@5: 0.6096897787665307  - hit@10: 0.6990483252997157
MRR in all data:  0.4996089153747799

 Model:  nli_distilroberta_base
median position: 2.0
hit@1: 0.4010629094055123  - hit@5: 0.6256334198492152  - hit@10: 0.7152391546162

In [4]:
median_rank = {}
hit_at_1 = {}
hit_at_5 = {}
hit_at_10 = {}
mrr_5 = {}
mrr_10 = {}
mrr_all = {}
fail_min = 10000
min_fail = {}

all_files =  os.listdir(directory)
sorted_files =  sorted(all_files)
for filename in sorted_files:
        if filename.endswith("ranking_results_8k.pkl"): 
            m = re.search('(.+?)_ranking_results_8k.pkl', filename)
            if m:
                model_name = m.group(1)
            df = pd.read_pickle(os.path.join(directory, filename))
            for idx, data in df.iterrows():
                for filename in os.listdir(directory):
                    if filename.endswith("_fail_file_name.txt") and filename.startswith(model_name):
                        with open(filename) as f:
                            lines = f.readlines()
                            fail_pct = (int(lines[0])/8091)*100
                min_fail[idx] = fail_pct
                name = idx.replace('_', ' ')
                median_rank[idx] = data['median hit pos all']
                hit_at_1[idx] = data['rank 1']
                hit_at_5[idx] = data['rank 5']
                hit_at_10[idx] = data['rank 10']
                mrr_5[idx] = data['mrr 5']
                mrr_10[idx] = data['mrr 10']
                mrr_all[idx] = data['mrr all']
            print("{} & {:.4f} & {:.4f} & {:.4f} & {:.4f} & {:.4f} & {:.4f} & {} & {:.2f} \\\\".format(name, hit_at_1[idx], hit_at_5[idx], hit_at_10[idx], mrr_5[idx], mrr_10[idx],mrr_all[idx], int(median_rank[idx]), fail_pct))


all MiniLM L12 & 0.4486 & 0.6670 & 0.7477 & 0.5307 & 0.5418 & 0.5504 & 2 & 55.14 \\
all MiniLM L6 & 0.4354 & 0.6568 & 0.7392 & 0.5183 & 0.5294 & 0.5380 & 2 & 56.46 \\
all distilroberta & 0.4699 & 0.6972 & 0.7754 & 0.5557 & 0.5662 & 0.5744 & 2 & 53.01 \\
all mpnet base & 0.4664 & 0.6872 & 0.7669 & 0.5503 & 0.5610 & 0.5692 & 2 & 53.36 \\
all roberta large & 0.4858 & 0.7081 & 0.7900 & 0.5700 & 0.5811 & 0.5886 & 2 & 51.42 \\
multi qa MiniLM L6 cos & 0.3662 & 0.5687 & 0.6541 & 0.4411 & 0.4526 & 0.4628 & 3 & 63.38 \\
multi qa distilbert cos & 0.3797 & 0.5874 & 0.6738 & 0.4583 & 0.4699 & 0.4795 & 3 & 62.03 \\
multi qa mpnet base cos & 0.3990 & 0.6097 & 0.6990 & 0.4784 & 0.4904 & 0.4996 & 3 & 60.10 \\
nli distilroberta base & 0.4011 & 0.6256 & 0.7152 & 0.4864 & 0.4984 & 0.5076 & 2 & 59.89 \\
nq distilbert base & 0.3260 & 0.5363 & 0.6187 & 0.4037 & 0.4149 & 0.4255 & 4 & 67.40 \\
paraphrase MiniLM L12 & 0.5289 & 0.7602 & 0.8338 & 0.6173 & 0.6273 & 0.6337 & 1 & 47.11 \\
paraphrase MiniLM L3 & 0.5

In [5]:
min_fail = dict(sorted(min_fail.items(), key=lambda item: item[1], reverse=True))
min_fail

{'nq_distilbert_base': 67.39587195649487,
 'multi_qa_MiniLM_L6_cos': 63.37906315659375,
 'stsb_roberta_large': 63.2183908045977,
 'multi_qa_distilbert_cos': 62.031887282165364,
 'multi_qa_mpnet_base_cos': 60.10381905821283,
 'nli_distilroberta_base': 59.893709059448774,
 'stsb_distilroberta_base': 58.94203435916451,
 'xlm_distilroberta_paraphrase': 58.620689655172406,
 'stsb_roberta_base': 57.940922012112225,
 'stsb_mpnet_base': 56.65554319614387,
 'all_MiniLM_L6': 56.45779260907181,
 'all_MiniLM_L12': 55.13533555802744,
 'all_mpnet_base': 53.355580274378944,
 'all_distilroberta': 53.00951674700284,
 'paraphrase_albert_base': 52.24323322209863,
 'all_roberta_large': 51.4151526387344,
 'paraphrase_mpnet': 50.63650970213818,
 'paraphrase_MiniLM_L3': 49.36349029786182,
 'paraphrase_MiniLM_L6': 48.62192559634161,
 'paraphrase_MiniLM_L12': 47.11407736991719,
 'paraphrase_distilroberta': 47.03992089976517,
 'paraphrase_TinyBERT_L6': 43.14670621678408}

In [6]:
median_rank = dict(sorted(median_rank.items(), key=lambda item: item[1], reverse=False))
median_rank

{'paraphrase_MiniLM_L12': 1.0,
 'paraphrase_MiniLM_L3': 1.0,
 'paraphrase_MiniLM_L6': 1.0,
 'paraphrase_TinyBERT_L6': 1.0,
 'paraphrase_distilroberta': 1.0,
 'all_MiniLM_L12': 2.0,
 'all_MiniLM_L6': 2.0,
 'all_distilroberta': 2.0,
 'all_mpnet_base': 2.0,
 'all_roberta_large': 2.0,
 'nli_distilroberta_base': 2.0,
 'paraphrase_albert_base': 2.0,
 'paraphrase_mpnet': 2.0,
 'stsb_distilroberta_base': 2.0,
 'stsb_mpnet_base': 2.0,
 'stsb_roberta_base': 2.0,
 'xlm_distilroberta_paraphrase': 2.0,
 'multi_qa_MiniLM_L6_cos': 3.0,
 'multi_qa_distilbert_cos': 3.0,
 'multi_qa_mpnet_base_cos': 3.0,
 'stsb_roberta_large': 3.0,
 'nq_distilbert_base': 4.0}

In [7]:
hit_at_1 = dict(sorted(hit_at_1.items(), key=lambda item: item[1], reverse=True))
hit_at_1

{'paraphrase_TinyBERT_L6': 0.5685329378321592,
 'paraphrase_distilroberta': 0.5296007910023482,
 'paraphrase_MiniLM_L12': 0.528859226300828,
 'paraphrase_MiniLM_L6': 0.5137807440365839,
 'paraphrase_MiniLM_L3': 0.5063650970213818,
 'paraphrase_mpnet': 0.49363490297861823,
 'all_roberta_large': 0.48584847361265604,
 'paraphrase_albert_base': 0.4775676677790137,
 'all_distilroberta': 0.46990483252997156,
 'all_mpnet_base': 0.4664441972562106,
 'all_MiniLM_L12': 0.4486466444197256,
 'all_MiniLM_L6': 0.4354220739092819,
 'stsb_mpnet_base': 0.43344456803856135,
 'stsb_roberta_base': 0.4205907798788778,
 'xlm_distilroberta_paraphrase': 0.41379310344827586,
 'stsb_distilroberta_base': 0.410579656408355,
 'nli_distilroberta_base': 0.4010629094055123,
 'multi_qa_mpnet_base_cos': 0.3989618094178717,
 'multi_qa_distilbert_cos': 0.3796811271783463,
 'stsb_roberta_large': 0.367816091954023,
 'multi_qa_MiniLM_L6_cos': 0.36620936843406254,
 'nq_distilbert_base': 0.3260412804350513}

In [8]:
hit_at_5 = dict(sorted(hit_at_5.items(), key=lambda item: item[1], reverse=True))
hit_at_5

{'paraphrase_TinyBERT_L6': 0.8059572364355456,
 'paraphrase_distilroberta': 0.764306019033494,
 'paraphrase_MiniLM_L12': 0.7602274131751329,
 'paraphrase_MiniLM_L6': 0.7497219132369299,
 'paraphrase_MiniLM_L3': 0.7336546780373254,
 'paraphrase_mpnet': 0.7318007662835249,
 'paraphrase_albert_base': 0.7167222840192807,
 'all_roberta_large': 0.7080706958348782,
 'all_distilroberta': 0.6971944135459152,
 'all_mpnet_base': 0.6871832900753924,
 'all_MiniLM_L12': 0.6670374490174268,
 'all_MiniLM_L6': 0.6567791373130639,
 'stsb_mpnet_base': 0.6517117785193425,
 'xlm_distilroberta_paraphrase': 0.6382400197750587,
 'stsb_roberta_base': 0.6353973550858979,
 'nli_distilroberta_base': 0.6256334198492152,
 'stsb_distilroberta_base': 0.6219255963416142,
 'multi_qa_mpnet_base_cos': 0.6096897787665307,
 'multi_qa_distilbert_cos': 0.5874428377209245,
 'stsb_roberta_large': 0.5693980966505995,
 'multi_qa_MiniLM_L6_cos': 0.5686565319490792,
 'nq_distilbert_base': 0.5362748733160302}

In [9]:
hit_at_10 = dict(sorted(hit_at_10.items(), key=lambda item: item[1], reverse=True))
hit_at_10

{'paraphrase_TinyBERT_L6': 0.8757879124953653,
 'paraphrase_distilroberta': 0.8399456185885552,
 'paraphrase_MiniLM_L12': 0.8337659127425534,
 'paraphrase_MiniLM_L6': 0.8258558892596712,
 'paraphrase_MiniLM_L3': 0.8183166481275491,
 'paraphrase_mpnet': 0.8117661599307873,
 'paraphrase_albert_base': 0.7956989247311828,
 'all_roberta_large': 0.7900135953528612,
 'all_distilroberta': 0.7754294895562971,
 'all_mpnet_base': 0.7669014954888147,
 'all_MiniLM_L12': 0.7477444073662094,
 'all_MiniLM_L6': 0.739216413298727,
 'stsb_mpnet_base': 0.735879372141886,
 'xlm_distilroberta_paraphrase': 0.7254974663206032,
 'stsb_roberta_base': 0.7200593251761216,
 'nli_distilroberta_base': 0.7152391546162402,
 'stsb_distilroberta_base': 0.7043628723272772,
 'multi_qa_mpnet_base_cos': 0.6990483252997157,
 'multi_qa_distilbert_cos': 0.6738351254480287,
 'stsb_roberta_large': 0.6575207020145841,
 'multi_qa_MiniLM_L6_cos': 0.6540600667408232,
 'nq_distilbert_base': 0.6187121493016933}

In [10]:
mrr_5 = dict(sorted(mrr_5.items(), key=lambda item: item[1], reverse=True))
mrr_5

{'paraphrase_TinyBERT_L6': 0.6604437028797389,
 'paraphrase_distilroberta': 0.619684423021461,
 'paraphrase_MiniLM_L12': 0.6172887570551617,
 'paraphrase_MiniLM_L6': 0.6043834713467612,
 'paraphrase_MiniLM_L3': 0.5932970790590336,
 'paraphrase_mpnet': 0.5845692745025322,
 'all_roberta_large': 0.569989288509865,
 'paraphrase_albert_base': 0.5683805050879561,
 'all_distilroberta': 0.5556606105549362,
 'all_mpnet_base': 0.550294565978659,
 'all_MiniLM_L12': 0.5307151979565762,
 'all_MiniLM_L6': 0.5183145882256014,
 'stsb_mpnet_base': 0.5159354014748897,
 'stsb_roberta_base': 0.5016520413628318,
 'xlm_distilroberta_paraphrase': 0.49788654060066806,
 'stsb_distilroberta_base': 0.49024842417501024,
 'nli_distilroberta_base': 0.48637786841346436,
 'multi_qa_mpnet_base_cos': 0.47836484983314886,
 'multi_qa_distilbert_cos': 0.45833230338235986,
 'stsb_roberta_large': 0.4426028921023362,
 'multi_qa_MiniLM_L6_cos': 0.4411197626992967,
 'nq_distilbert_base': 0.4037222428212422}

In [11]:
mrr_10 = dict(sorted(mrr_10.items(), key=lambda item: item[1], reverse=True))
mrr_10

{'paraphrase_TinyBERT_L6': 0.6699910541401057,
 'paraphrase_distilroberta': 0.6298310095677526,
 'paraphrase_MiniLM_L12': 0.6272663335510935,
 'paraphrase_MiniLM_L6': 0.6146883270261081,
 'paraphrase_MiniLM_L3': 0.6047262488400775,
 'paraphrase_mpnet': 0.595281451269586,
 'all_roberta_large': 0.5810711195861354,
 'paraphrase_albert_base': 0.5789457912279571,
 'all_distilroberta': 0.5661583122929057,
 'all_mpnet_base': 0.5609853590016746,
 'all_MiniLM_L12': 0.5417722611641774,
 'all_MiniLM_L6': 0.529368855851987,
 'stsb_mpnet_base': 0.5271731965558442,
 'stsb_roberta_base': 0.5130925406045921,
 'xlm_distilroberta_paraphrase': 0.5096282759797792,
 'stsb_distilroberta_base': 0.5013498733650769,
 'nli_distilroberta_base': 0.49835153894293804,
 'multi_qa_mpnet_base_cos': 0.49036863416730103,
 'multi_qa_distilbert_cos': 0.4698863915030043,
 'stsb_roberta_large': 0.4544504181600962,
 'multi_qa_MiniLM_L6_cos': 0.4525911114250024,
 'nq_distilbert_base': 0.41486611618239433}

In [12]:
mrr_all = dict(sorted(mrr_all.items(), key=lambda item: item[1], reverse=True))
mrr_all

{'paraphrase_TinyBERT_L6': 0.6749065745212289,
 'paraphrase_distilroberta': 0.6360536672638312,
 'paraphrase_MiniLM_L12': 0.6336502420932028,
 'paraphrase_MiniLM_L6': 0.6211562284389277,
 'paraphrase_MiniLM_L3': 0.6115218888683002,
 'paraphrase_mpnet': 0.602188949781251,
 'all_roberta_large': 0.5886092053186419,
 'paraphrase_albert_base': 0.586448437441523,
 'all_distilroberta': 0.5743539984492478,
 'all_mpnet_base': 0.5691792013584119,
 'all_MiniLM_L12': 0.550377342709509,
 'all_MiniLM_L6': 0.5380214519774836,
 'stsb_mpnet_base': 0.5356094035194916,
 'stsb_roberta_base': 0.5224353969410293,
 'xlm_distilroberta_paraphrase': 0.5187406713726074,
 'stsb_distilroberta_base': 0.510936609435869,
 'nli_distilroberta_base': 0.5075846869188708,
 'multi_qa_mpnet_base_cos': 0.4996089153747799,
 'multi_qa_distilbert_cos': 0.4795275412465367,
 'stsb_roberta_large': 0.4642690105047136,
 'multi_qa_MiniLM_L6_cos': 0.46279232277470167,
 'nq_distilbert_base': 0.4254914211642071}